In [ ]:
! pip install joblib

In [ ]:
! pip install pandas

In [ ]:
! pip install numpy

In [ ]:
! pip install lightgbm

In [ ]:
! pip install sklearn

In [ ]:
! pip install statistics

In [1]:
import pandas as pd
import numpy as np
import joblib as jl

In [2]:
#드라이브 미 사용시 코드 블록 제거
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
#파일경로
%cd /content/drive/MyDrive/통신망해커톤/Q1_data

/content/drive/MyDrive/통신망해커톤/Q1_data


데이터 불러오기

In [5]:
train = pd.read_csv('Q1_tr_최종전처리.csv')
test = pd.read_csv('Q1_te_최종전처리.csv')

In [6]:
#0~12 클래스만 남기기
idx = train[train['uenomax']>12].index
train.drop(idx, axis=0, inplace=True)

In [7]:
X = train.drop(['uenomax', 'ru_id'], axis=1)
y = train['uenomax']

In [8]:
test = test.drop(['ru_id'], axis=1)

In [9]:
#스케일링
from sklearn.preprocessing import MinMaxScaler

scaler_ = MinMaxScaler()
scaler_.fit(X)
X = scaler_.transform(X)
test = scaler_.transform(test)

모델 불러오기

In [10]:
model1 = jl.load('Q1_lgbm12optuna_submitmae0.4089.pkl')
model2 = jl.load('Q1_lgbm12optuna_submitmae0.4087.pkl')
model3 = jl.load('Q1_lgbm12optuna_submitmae0.4094.pkl')

pred1

In [11]:
pred1 = model1.predict(test)

In [12]:
rst1 = pd.read_csv('Q1_test.csv', usecols=['datetime', 'ru_id'])
rst1['uenomax'] = pred1

In [13]:
def ppr_label(df):
    df = df[['datetime', 'ru_id', 'uenomax']]  # 레이블 추출
    df_pivot = df.pivot_table(index='datetime', columns='ru_id', values='uenomax', fill_value=0)  # 데이터 재구성
    return df_pivot

rst1 = ppr_label(rst1)

In [14]:
rst1.reset_index(inplace=True)

pred2

In [15]:
pred2 = model2.predict(test)

In [16]:
rst2 = pd.read_csv('Q1_test.csv', usecols=['datetime', 'ru_id'])
rst2['uenomax'] = pred2

In [17]:
def ppr_label(df):
    df = df[['datetime', 'ru_id', 'uenomax']]  # 레이블 추출
    df_pivot = df.pivot_table(index='datetime', columns='ru_id', values='uenomax', fill_value=0)  # 데이터 재구성
    return df_pivot

rst2 = ppr_label(rst2)

In [18]:
rst2.reset_index(inplace=True)

pred3

In [19]:
pred3 = model3.predict(test)

In [20]:
rst3 = pd.read_csv('Q1_test.csv', usecols=['datetime', 'ru_id'])
rst3['uenomax'] = pred3

In [21]:
def ppr_label(df):
    df = df[['datetime', 'ru_id', 'uenomax']]  # 레이블 추출
    df_pivot = df.pivot_table(index='datetime', columns='ru_id', values='uenomax', fill_value=0)  # 데이터 재구성
    return df_pivot

rst3 = ppr_label(rst3)

In [22]:
rst3.reset_index(inplace=True)

voting

In [23]:
rstlist = [rst1,rst2,rst3]

In [24]:
import statistics

finaldf = pd.DataFrame({'datetime':rstlist[0]['datetime'],
                        'BaseStationB':None,
                        'BaseStationJ':None})

for c in range(2):
  c += 1
  for i in range(17181):
    votes = []
    for df in rstlist:
      v = df.iloc[i, c]
      votes.append(v)
    if len(set(votes)) == 3: #세 모델이 모두 다르게 예측한 경우
      major = votes[1] #가장 mae가 낮은 두 번째 모델의 예측값
    else:
      major = statistics.mode(votes) #최빈값(hard voting)
    finaldf.iloc[i,c] = major

In [25]:
finaldf

,datetime,BaseStationB,BaseStationJ
0,2023-05-19 00:00:00,1,1
1,2023-05-19 00:05:00,2,2
2,2023-05-19 00:10:00,2,2
3,2023-05-19 00:15:00,2,1
4,2023-05-19 00:20:00,2,2
...,...,...,...
17176,2023-07-17 15:30:00,2,2
17177,2023-07-17 15:35:00,1,2
17178,2023-07-17 15:40:00,2,2
17179,2023-07-17 15:45:00,3,1


In [26]:
finaldf.to_csv('Q1_중꺾마_최종제출파일.csv', index=False)